In [1]:
# ============================
#         IMPORTAÇÕES
# ============================
from __future__ import annotations
from dataclasses import dataclass
from enum import Enum
from typing import List, Optional, Tuple, Dict
from datetime import datetime, timedelta
from collections import deque, defaultdict
import random
import pandas as pd
import time
# ============================
#        MODELOS BÁSICOS
# ============================

from sprintTresContent.Criticidade import Criticidade
from sprintTresContent.Setor import Setor
from sprintTresContent.InsumoHospitalar import InsumoHospitalar
from sprintTresContent.ConsumoAggregado import ConsumoAggregado

# ============================
#     ESTRUTURAS DE DADOS
# ============================
from sprintTresContent.FilaConsumoHospitalar import FilaConsumoHospitalar
from sprintTresContent.PilhaConsultasHospitalar import PilhaConsultasHospitalar

# ============================
#     BUSCAS E ORDENAÇÕES
# ============================
from sprintTresContent.BuscaInsumosHospitalares import BuscaInsumosHospitalares
from sprintTresContent.OrdenacaoInsumosHospitalares import OrdenacaoInsumosHospitalares

# ============================
#   SISTEMA PRINCIPAL (DEMO)
# ============================
from sprintTresContent.SistemaConsumoHospitalar import SistemaConsumoHospitalar
# ============================
#            MAIN
# ============================
print("🏥 SISTEMA DE GERENCIAMENTO DE INSUMOS HOSPITALARES")
print("Estruturas de Dados e Algoritmos aplicados ao contexto médico\n")
sistema = SistemaConsumoHospitalar()
sistema.gerar_dados_hospitalares_simulados(20)
sistema.demonstrar_fila_hospitalar()
sistema.demonstrar_pilha_hospitalar()
sistema.demonstrar_buscas_hospitalares()
sistema.demonstrar_ordenacao_hospitalar()
sistema.gerar_relatorio_hospitalar()
sistema.imprimir_justificativas()

def consumo_dataframe(sistema: SistemaConsumoHospitalar):
    """Cria um DataFrame com os registros de consumo hospitalar"""
    dados = []
    for data, insumo, qtd, responsavel in sistema.fila_consumo.items:
        dados.append({
            "Data": data.strftime('%d/%m/%Y %H:%M'),
            "Nome": insumo.nome,
            "Código": insumo.codigo,
            "Quantidade Consumida": qtd,
            "Responsável": responsavel,
            "Setor": insumo.setor.value,
            "Criticidade": insumo.criticidade.value,
            "Validade": insumo.validade.strftime('%d/%m/%Y'),
            "Lote": insumo.lote,
            "Fornecedor": insumo.fornecedor
        })
    df = pd.DataFrame(dados)
    return df

def medir_tempo(func, *args, **kwargs):
    """Wrapper para medir tempo de execução de um algoritmo"""
    inicio = time.time()
    resultado = func(*args, **kwargs)
    fim = time.time()
    print(f"Tempo de execução de {func.__name__}: {(fim - inicio):.6f} segundos")
    return resultado

# Exemplo de uso:
sistema = SistemaConsumoHospitalar()
sistema.gerar_dados_hospitalares_simulados(20)

# DataFrame de consumo
df_consumo = consumo_dataframe(sistema)
display(df_consumo.head())

# Medindo performance dos algoritmos
medir_tempo(sistema.ordenacao.merge_sort, sistema.insumos, 'quantidade')
medir_tempo(sistema.ordenacao.quick_sort, sistema.insumos, 'validade')
medir_tempo(sistema.busca.busca_sequencial_por_codigo, sistema.insumos, "SER001")
medir_tempo(sistema.busca.busca_binaria_por_nome, sorted(sistema.insumos, key=lambda x: x.nome.lower()), "Luva Nitrilo")
# ============================

🏥 SISTEMA DE GERENCIAMENTO DE INSUMOS HOSPITALARES
Estruturas de Dados e Algoritmos aplicados ao contexto médico


DEMONSTRAÇÃO DA FILA - REGISTRO DE CONSUMO HOSPITALAR (FIFO)

=== REGISTROS DE CONSUMO EM ORDEM CRONOLÓGICA ===
07/09/2025 07:55: Seringa 10ml (4 unidades) - Responsável: Dr. Silva
07/09/2025 02:55: Luva Nitrilo (10 unidades) - Responsável: Dr. Silva
08/09/2025 05:55: Máscara N95 (4 unidades) - Responsável: Dr. Costa
08/09/2025 12:55: Cateter Venoso (2 unidades) - Responsável: Dr. Costa
08/09/2025 23:55: Gaze Estéril (8 unidades) - Responsável: Dr. Santos
09/09/2025 11:55: Soro Fisiológico (4 unidades) - Responsável: Dr. Santos
10/09/2025 07:55: Dipirona 500mg (6 unidades) - Responsável: Dr. Silva
10/09/2025 03:55: Morfina 10mg (4 unidades) - Responsável: Dr. Santos
11/09/2025 00:55: Oxímetro Digital (9 unidades) - Responsável: Enf. Ana
11/09/2025 14:55: Termômetro Digital (10 unidades) - Responsável: Dr. Costa
12/09/2025 03:55: Atadura Elástica (4 unidades) - Responsável:

,Data,Nome,Código,Quantidade Consumida,Responsável,Setor,Criticidade,Validade,Lote,Fornecedor
0,07/09/2025 01:55,Seringa 10ml,SER001,4,Dr. Santos,Emergência,Alta,01/08/2026,LT1359,MedSupply
1,07/09/2025 09:55,Luva Nitrilo,LUV001,7,Dr. Silva,Farmácia,Crítica,08/04/2026,LT4220,BioMed
2,08/09/2025 02:55,Máscara N95,MAS001,8,Dr. Silva,Pediatria,Crítica,21/09/2027,LT2750,MedSupply
3,08/09/2025 11:55,Cateter Venoso,CAT001,2,Enf. Maria,Cirurgia,Alta,19/09/2026,LT4921,Cirúrgica Brasil
4,09/09/2025 12:55,Gaze Estéril,GAZ001,2,Dr. Costa,Cirurgia,Média,17/02/2026,LT6441,Hospitalar SP


Tempo de execução de merge_sort: 0.000072 segundos
Tempo de execução de quick_sort: 0.000027 segundos
Tempo de execução de busca_sequencial_por_codigo: 0.000006 segundos
Tempo de execução de busca_binaria_por_nome: 0.000004 segundos


9